In [5]:
from pystempel import Stemmer
import csv
import ipywidgets as widgets
from IPython.display import display
import random
import spacy
from PyQt5.QtWidgets import QApplication, QVBoxLayout, QWidget, QPushButton, QLineEdit, QLabel
from PyQt5.QtGui import QScreen, QGuiApplication
from PyQt5.QtCore import Qt
import sys
import os

In [9]:
with open('interactions.csv', 'w') as file:
    dialog = csv.writer(file, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    dialog.writerow(['Witaj', 'Witaj! W czym mogę ci poomóc?'])
    dialog.writerow(['Jak się masz?', 'Dobrze. Dziękuję, że pytasz. A ty?'])
    dialog.writerow(['Czym jesteś?', 'Jestem chatbotem.'])
    dialog.writerow(['Jakie masz zainteresowania?', 'Lubię rozmawiać z ludźmi.'])
    dialog.writerow(['Dziękuję', "Proszę bardzo. Cieszę się, że mogłem pomóc."])
    dialog.writerow(['Do widzenia', 'Do widzenia! Miłego dnia!'])
    dialog.writerow(['Jak minął ci poranek?', 'Był całkiem niezły, dziękuję!'])
    dialog.writerow(['Co lubisz jeść?', 'Uwielbiam pizzę!'])
    dialog.writerow(['Polecisz jakąś książkę?', "Polecam 'Wiedźmina'!"])
    dialog.writerow(['Kim jesteś?', 'Jestem wirtualnym asystentem do rozmów.'])
    dialog.writerow(['Lubię programowanie', 'Ja też bardzo lubię programować w Pythonie!'])
    dialog.writerow(["Jakie jest Twoje ulubione miasto?", "Uwielbiam atmosferę Krakowa."])
    dialog.writerow(["Lubisz sporty ekstremalne?", "Tak, szczególnie skoki na bungee."])
    dialog.writerow(["Czy potrafisz grać na instrumencie?", "Chciałbym nauczyć się gry na gitarze."])
    dialog.writerow(["Jaka kuchnia świata ci najbardziej smakuje?", "Kocham kuchnię włoską."])
    dialog.writerow(["Masz jakieś zwierzęta domowe?", "Niestety, nie mam, ale bardzo lubię psy."])
    dialog.writerow(["Jaki jest twój ulubiony język programowania?", "Oczywiście Python!"])
    dialog.writerow(["Gdzie najchętniej spędzasz wakacje?", "Nad morzem w ciepłym klimacie."])
    dialog.writerow(["Znasz jakieś dowcipy?", "Niektórzy mówią, że sama jestem jednym wielkim żartem."])
    dialog.writerow(["Czy lubisz książki fantasy?", "Tak, szczególnie Tolkiena."])
    dialog.writerow(["Jakiej muzyki słuchasz najczęściej?", "Głównie rock i pop."])
    dialog.writerow(["Czy masz ulubionego sportowca?", "Podziwiam Roberta Lewandowskiego."])
    dialog.writerow(["Znasz jakieś języki obce?", "Trochę angielskiego i niemieckiego."])
    dialog.writerow(["Jakie są twoje cele na przyszłość?", "Pomagać ludziom w rozmowach."])
    dialog.writerow(["Co cię najbardziej inspiruje?", "Ludzka kreatywność i inteligencja."])
    dialog.writerow(["Lubisz gotować?", "Tak, sprawia mi to przyjemność."])
    dialog.writerow(["Masz ulubioną porę roku?", "Uwielbiam letnie wieczory."])
    dialog.writerow(["Czy czytasz gazety codzienne?", "Czasem przeglądam wiadomości online."])
    dialog.writerow(["Jaki jest twój ulubiony kolor?", "Niebieski, przypomina mi niebo."])
    dialog.writerow(["Jak lubisz spędzać wolny czas?", "Lubię przeglądać memy w internecie."])
    dialog.writerow(["Czy lubisz oglądać filmy?", "Bardzo, szczególnie science fiction."])
    dialog.writerow(["Jakie jest twoje największe marzenie?", "Odkryć sens istnienia."])
    dialog.writerow(["Znaleźć miłość to ważna sprawa?", "Dla wielu ludzi najważniejsza."])
    dialog.writerow(["Czy potrafisz coś narysować?", "Nie mam umiejętności plastycznych."])
    dialog.writerow(["Jak radzisz sobie ze stresem?", "Staram się oddychać głęboko i myśleć pozytywnie."])
    dialog.writerow(["Czy jeździsz na rowerze?", "Uwielbiam długie wycieczki rowerowe."])
    dialog.writerow(["Jaki jest twój ulubiony film?", "Trudno wybrać, ale 'Incepcja' jest wysoko na liście."])
    dialog.writerow(["Znasz jakieś sztuczki magiczne?", "Niestety, żadnych."])
    dialog.writerow(["Czy lubisz występy stand-upowe?", "Tak, czasem są naprawdę zabawne."])
    dialog.writerow(["Gdzie chciałbyś teraz pojechać?", "Japonia brzmi ciekawie."])
    dialog.writerow(["Jaka jest najlepsza rada, jaką kiedykolwiek otrzymałeś?", "Ucz się na błędach i próbuj ponownie."])
    dialog.writerow(["Lubisz zdobywać nowe umiejętności?", "Tak, to zawsze ciekawa przygoda."])
    dialog.writerow(["Czy grasz w gry komputerowe?", "Tak, lubię gry logiczne i RPG."])
    dialog.writerow(["Jakie jest twoje największe osiągnięcie?", "Udzielanie sensownych odpowiedzi w czasie rzeczywistym."])
    dialog.writerow(["Czy lubisz zaskakiwać ludzi?", "Pozytywne niespodzianki są świetne."])
    dialog.writerow(["Co cenisz w przyjaźni?", "Szczerość i wzajemne wsparcie."])
    dialog.writerow(["Masz jakieś ulubione cytaty?", "Życie jest krótkie, śmiej się często."])
    dialog.writerow(["Czy lubisz podejmować ryzyko?", "Zależy od sytuacji, ale bywam ostrożny."])
    dialog.writerow(["Jaki jest twój stosunek do kawy?", "Lubię zapach, ale piję ją rzadko."])
    dialog.writerow(["Możesz opisać siebie jednym słowem?", "Ciekawski."])
    dialog.writerow(["Co robisz, gdy się nudzisz?", "Szukam nowych informacji i uczę się."])

# Mam nadzieję, że ilość przykładowych promptów nie wpływa na ocenę, bo skupiłem się bardziej na oknach, a promptów dałem tylko kilka przykładowych, żeby cokolwiek sobie sprawdzić
# Następnie wygenerowałem do 6 moich ręcznych kilka z GitHub Copilot in-line generation

In [ ]:
#PyStempel
stemmer = Stemmer.polimorf()
class Chatbot():
    def __init__(self, dialog_file):
        self.dialog_file = dialog_file
    
    def make_dialog_tuples(self):
        dialog = []
        with open(self.dialog_file, newline='') as file:
            csv_reader = csv.reader(file, delimiter=',', quotechar='"')
            for row in csv_reader:
                dialog.append((row[0], row[1]))
        return dialog
    
    def split_and_clean(self, sentence):
        interpoint = ',.?!\'":\\;@#$%^&*/+-=_`~<>{}()[]'    #Rozszerzyłem listę znaków
        if len(sentence[0])>1:          #Warunek upewniający się, czy to jest zdanie, czy tylko jedno słowo
            for word in sentence:       #Zmieniłem sprawdzanie znaków, teraz usuwa znaki nawet ze środka
                if word[0] in interpoint:
                    word = word[1:]
                if word[-1] in interpoint:
                    word = word[:-1]
        else:
            if sentence[0] in interpoint:
                sentence = sentence[1:]
            if sentence[-1] in interpoint:
                sentence = sentence[:-1]
        for inter in interpoint:
            if inter not in '\'':
                sentence.replace(inter, '')
        sentence_tokenized = [word.lower() for word in sentence.split()]
        sentence_tokenized_clean = []
        for word in sentence_tokenized:
            sentence_tokenized_clean.append(stemmer(word))
        return sentence_tokenized_clean
    
    def find_matching(self, input_prompt, dialog):
        answer = 'Nie posiadam na to odpowiedzi.'
        intersection_len_req = 0
        input_prompt = set(self.split_and_clean(input_prompt.lower()))  #Wyrzuciłem poza for, bo nie ma powodu wykonywać się wiele razy
        for index, pair in enumerate(dialog):
            match = set(self.split_and_clean(pair[0]))
            intersection_len = len(set.intersection(input_prompt, match))
            if intersection_len > intersection_len_req:
                answer = index
                intersection_len_req = intersection_len
            if intersection_len == intersection_len_req and intersection_len != 0:
                if random.randint(0, 1) == 1:
                    answer = index
        
        display(f'\tPrompt: {input_prompt}')
        if answer != 'Nie posiadam na to odpowiedzi.':
            answer = dialog[answer][1]
        return answer
    
    def run(self):
        dialog = self.make_dialog_tuples()
        input_prompt = widgets.Text(
            value='',
            placeholder='Witaj',
            description="Prompt:",
        )
        submit_button = widgets.Button(description='Submit')
        def on_submit(b):
            if not input_prompt:
                display('Puste')
            else:
                answer = self.find_matching(input_prompt.value, dialog)
                display(f'Response: {answer}')
        submit_button.on_click(on_submit)
        display(input_prompt, submit_button)

In [10]:
#Spacy
class Chatbot():
    def __init__(self, dialog_file):
        self.dialog_file = dialog_file
        self.nlp = spacy.load("pl_core_news_md")
    
    def make_dialog_tuples(self):
        dialog = []
        with open(self.dialog_file, newline='') as file:
            csv_reader = csv.reader(file, delimiter=',', quotechar='"')
            for row in csv_reader:
                dialog.append((row[0], row[1]))
        return dialog
    
    def split_and_clean(self, sentence):
        doc = self.nlp(sentence)
        return [token.lemma_.lower() for token in doc if token.is_alpha]
    
    def find_matching(self, input_prompt, dialog):
        answer = 'Nie posiadam na to odpowiedzi.'
        intersection_len_req = 0
        input_prompt = set(self.split_and_clean(input_prompt))  #Wyrzuciłem poza for, bo nie ma powodu wykonywać się wiele razy
        for index, pair in enumerate(dialog):
            match = set(self.split_and_clean(pair[0]))
            intersection_len = len(set.intersection(input_prompt, match))
            if intersection_len > intersection_len_req:
                answer = index
                intersection_len_req = intersection_len
            if intersection_len == intersection_len_req and intersection_len != 0:
                if random.randint(0, 1) == 1:
                    answer = index
        
        display(f'\tPrompt: {input_prompt}')
        if answer != 'Nie posiadam na to odpowiedzi.':
            answer = dialog[answer][1]
        return answer
    
    def run(self):
        dialog = self.make_dialog_tuples()
        input_prompt = widgets.Text(
            value='',
            placeholder='Witaj',
            description="Prompt:",
        )
        submit_button = widgets.Button(description='Submit')
        def on_submit(b):
            if not input_prompt:
                display('Puste')
            else:
                answer = self.find_matching(input_prompt.value, dialog)
                display(f'Response: {answer}')
        submit_button.on_click(on_submit)
        display(input_prompt, submit_button)

In [19]:
#QT_App
class Chatbot():
    def __init__(self, dialog_file):
        self.dialog_file = dialog_file
        self.nlp = spacy.load("pl_core_news_md")
    
    def make_dialog_tuples(self):
        dialog = []
        with open(self.dialog_file, newline='') as file:
            csv_reader = csv.reader(file, delimiter=',', quotechar='"')
            for row in csv_reader:
                dialog.append((row[0], row[1]))
        return dialog
    
    def split_and_clean(self, sentence):
        doc = self.nlp(sentence)
        return [token.lemma_.lower() for token in doc if token.is_alpha]
    
    def find_matching(self, input_prompt, dialog):
        answer = 'Nie posiadam na to odpowiedzi.'
        intersection_len_req = 0
        input_prompt = set(self.split_and_clean(input_prompt))
        for index, pair in enumerate(dialog):
            match = set(self.split_and_clean(pair[0]))
            intersection_len = len(set.intersection(input_prompt, match))
            if intersection_len > intersection_len_req:
                answer = index
                intersection_len_req = intersection_len
            if intersection_len == intersection_len_req and intersection_len != 0:
                if random.randint(0, 1) == 1:
                    answer = index
        
        display(f'\tPrompt: {input_prompt}')
        if answer != 'Nie posiadam na to odpowiedzi.':
            answer = dialog[answer][1]
        return answer
    
    def ask(self, prompt):
        dialog = self.make_dialog_tuples()
        answer = self.find_matching(prompt, dialog)
        return answer

class ChatbotApp(QWidget):
    def __init__(self, dialog_file):
        super().__init__()
        self.setWindowTitle("ChatBot")
        self.setGeometry(100, 100, 400, 200)
        self.chatbot = Chatbot(dialog_file)

        self.layout = QVBoxLayout()

        self.input_field = QLineEdit(self)
        self.input_field.setPlaceholderText("Podaj promopt")
        self.layout.addWidget(self.input_field)

        self.button = QPushButton("Wyślij", self)
        self.button.clicked.connect(self.submit_onClick)
        self.layout.addWidget(self.button)

        self.label_prompt = QLabel("Prompt:", self)
        self.layout.addWidget(self.label_prompt)

        self.label_decomposition = QLabel("Rozkład:", self)
        self.layout.addWidget(self.label_decomposition)

        self.output_field = QLabel("Odpowiedź", self)
        self.layout.addWidget(self.output_field)

        quit_button = QPushButton("Wyjdź", self)
        quit_button.clicked.connect(self.close_application)
        self.layout.addWidget(quit_button)

        self.setLayout(self.layout)
        self.centerWindow()

    def centerWindow(self):
        screen_geometry = QApplication.primaryScreen().availableGeometry()
        window_geometry = self.frameGeometry()
        window_geometry.moveCenter(screen_geometry.center())
        self.move(window_geometry.topLeft())

    def close_application(self):
        QApplication.quit()

    def submit_onClick(self):
        response = self.chatbot.ask(self.input_field.text())
        self.label_prompt.setText(f"Prompt: {self.input_field.text()}")
        self.label_decomposition.setText(f"Rozkład: {self.chatbot.split_and_clean(self.input_field.text())}")
        self.output_field.setText(f"Odpowiedź: {response}")
        self.input_field.clear()

In [20]:
#Uruchomienie do aplikacji QT
#QGuiApplication.setAttribute(Qt.AA_EnableHighDpiScaling, True) #Również brało udział w skalowaniu
#QGuiApplication.setAttribute(Qt.AA_UseHighDpiPixmaps, True)    #Również brało udział w skalowaniu

#os.environ['QT_SCALE_FACTOR'] = '1.5' #Potrzebowałem wyskalować aplikację

app = QApplication(sys.argv)
window = ChatbotApp("interactions.csv")
window.show()
try:
    app.exec_()
finally:
    del app             #Zamykanie w naturalny sposób z nieznanego powodu wyrzuca błąd, dopiero uczę się QT

#Wiem, że Aplikację QT powinno się robić w QML, a nie na widżetach, ale nie chciałem już tutaj więcej plików tworzyć dla takiego prostego przykładu.
#Przy bardziej zaawansowanych zadaniach w przyszłości pewnie przerzucę się na QML

Dropped Escape call with ulEscapeCode : 0x03007703


"\tPrompt: {'program', 'na', 'witaj', 'o', 'świat', 'ten', 'wśród'}"

In [ ]:
text = "To jest tekst"
nlp = spacy.load("pl_core_news_md")
doc = nlp(text)

()

In [ ]:
#Uruchomienie do aplikacji na widżetach jupyer
chatbot = Chatbot('interactions.csv')
chatbot.run()

Text(value='', description='Prompt:', placeholder='Witaj')

Button(description='Submit', style=ButtonStyle())

"\tPrompt: {'witaj'}"

'Response: Witaj! W czym mogę ci poomóc?'

"\tPrompt: {'dziękować'}"

'Response: Proszę bardzo. Cieszę się, że mogłem pomóc.'